In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv

load_dotenv(".env")

tx_hash="0x8fe7d468383ab1e9954fbd02ebde849f58cd8748b761000ce65b11709bc054da"

INFURA_API_KEY = os.getenv("INFURA_API_KEY")
# 连接到Infura的以太坊节点
infura_url = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
ethers = Web3(Web3.HTTPProvider(infura_url))
receipt=ethers.eth.get_transaction_receipt(tx_hash) 
print(ethers.to_json(receipt))

In [ ]:
import json

# 连接到Infura节点
web3 = Web3(Web3.HTTPProvider(f'https://mainnet.infura.io/v3/{INFURA_API_KEY}'))

# 事件的ABI
TRANSFER_EVENT_ABI = {
    "anonymous": False,
    "inputs": [
        {"indexed": True, "name": "from", "type": "address"},
        {"indexed": True, "name": "to", "type": "address"},
        {"indexed": False, "name": "value", "type": "uint256"}
    ],
    "name": "Transfer",
    "type": "event"
}

# 将事件ABI转换为event签名hash
event_signature_hash = web3.keccak(text='Transfer(address,address,uint256)').hex()

# 获取交易日志的函数
def get_transaction_logs(tx_hash):
    try:
        receipt = web3.eth.get_transaction_receipt(tx_hash)
        if receipt and receipt['logs']:
            logs = [log for log in receipt['logs'] if log['topics'][0].hex() == event_signature_hash]
            return logs
        else:
            return None
    except Exception as e:
        print(f"Error fetching transaction receipt: {e}")
        return None

# 解码日志的函数
def decode_logs(logs):
    decoded_logs = []
    for log in logs:
        if log['topics'][0].hex() == event_signature_hash:
            # 解析 Transfer 事件
            from_address = Web3.to_checksum_address("0x" + log['topics'][1].hex()[-40:])
            to_address = Web3.to_checksum_address("0x" + log['topics'][2].hex()[-40:])
            value = int(log['data'].hex(), 16)
            decoded_logs.append(f"Transfer Event - From: {from_address}, To: {to_address}, Value: {value}")
    return decoded_logs

# 替换为你要查询的交易哈希

# 获取并解码日志
logs = get_transaction_logs(tx_hash)
if logs:
    decoded_logs = decode_logs(logs)
    print(f"Decoded Logs: {json.dumps(decoded_logs, indent=2)}")
else:
    print("No logs found for this transaction.")

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_address_tag(address):
    """
    Get the tag for a given Ethereum address from Etherscan.

    :param address: Ethereum address.
    :return: Address tag or None if not found.
    """
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tag_element = soup.find("span", {"class": "u-label--primary"})
        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    elif response.status_code == 403:
        return "Access denied (403). Try using a proxy or reduce the request frequency."
    else:
        print("Error:", response.status_code)
        return None

# Example usage
address = "0xc0ffeebabe5d496b2dde509f9fa189c25cf29671"  # Replace with the actual Ethereum address
tag = get_address_tag(address)

if tag:
    print(f"The tag for address {address} is '{tag}'.")
else:
    print("Address tag not found.")

In [ ]:
# 要获取标签，可能需要额外的etherscan高级API或其他第三方服务
label_url = f'https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85&address={address}&tag=latest&apikey={os.getenv("ETHERSCAN_API_KEY")}'
label_response = requests.get(label_url)
label_data = label_response.json()

if label_data['status'] == '1':
    label = label_data['result']
    print(f"Address Label: {label}")
else:
    print(f"Error fetching label: {label_data['message']}")

In [ ]:
contract_info_url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&apikey={os.getenv("ETHERSCAN_API_KEY")}'
contract_info_response = requests.get(contract_info_url)
contract_info = contract_info_response.json().get('result')[0]
print(contract_info)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_address_tag(address):
    url = f"https://etherscan.io/address/{address}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    # 发送HTTP GET请求到Etherscan地址页面
    response = requests.get(url, headers=headers)
    
    # 检查请求是否成功
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 查找包含标签的HTML元素
        tag_element = soup.find('span', {'class': 'hash-tag text-truncate lh-sm my-n1'})
        
        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    else:
        return f"Failed to retrieve page, status code: {response.status_code}"

# 示例地址
address = "0x742d35Cc6634C0532925a3b844Bc454e4438f44e"
tag = get_address_tag(address)
print(f"Address: {address}, Tag: {tag}")